In [ ]:
import import_ipynb
import random
import numpy as np 
import tensorflow as tf
from tensorflow.python.client import device_lib
from word_sequence import WordSequence

In [ ]:
VOCAB_SIZE_THRESHOLD_CPU = 50000

In [12]:
# Getting a list of GPU
def _get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return local_device_protos
    #return [x.name for x in local_device_protos if x.device_type == 'GPU' ]
    

In [ ]:

#Deciding the use of GPU/CPU based on the data-size and memory size
#If the data-size is too large, it will cosume a lof of GPU-memory
def _get_embed_device(vocab_size):
    gpus = _get_abailable_gpus()
    if not gpus or vocab_size> VOCAB_SIZE_THRESHOLD_CPU:
        return "/cpu:0"
    return "/gpu:0"

In [ ]:
# transform sentences to vector using word_sequence
def trans_sent(sentence, ws, max_len = None, add_end = False):
    encoded = ws.transform(
        sentence,
        max_len = max_len if max_len is not None else len(sentence))
    encoded_len = len(sentence) + (1 if add_end else 0)
    if encoded_len > len(encoded):
        encoded_len = len(encoded)
    
    return encoded, encoded_len

In [ ]:
def batch_flow(data, ws, batch_sizes, raw = False, add_end = True):
    all_data = list(zip(*data))
    
    if isinstance(ws,(list,tuple)):
        assert len(ws) == len(data),'The lengh of word sequence is not equal to the length of data while word sequence is list or tuple'
        
    if isinstance(add_end, bool):
        add_end = [add_end] * len(data)
    else:
        assert(isinstance(add_end,(list,tuple))),"add_end must be bool, list or tuple"
        assert len(add_end) == len(data), "The length of add_end should be equal as data when it is not bool"
        
    
    multiple = 2
    if raw:
        multiple = 3
        
    for d in data_batch:
            for j in range(len(data)):
                if isinstance(ws, (list, tuple)):
                    w = ws[j]
                else:
                    w = ws

                #添加结束标记（结尾）
                line = d[j]
                if add_end[j] and isinstance(line, (tuple, list)):
                    line = list(line) + [WordSequence.END_TAG]
                if w is not None:
                    x, xl = transform_sentence(line, w, max_lens[j], add_end[j])
                    batches[j * mul].append(x)
                    batches[j * mul + 1].append(xl)
                else:
                    batches[j * mul].append(line)
                    batches[j * mul + 1].append(line)
                if raw:
                    batches[j * mul + 2].append(line)
        batches = [np.asarray(x) for x in batches]
        yield batches

In [ ]:
def batch_flow_bucket(data, ws, batch_size, raw=False, add_end=True,
                      n_bucket=5, bucket_ind=1, debug=False):

    all_data = list(zip(*data))
    lengths = sorted(list(set([len(x[bucket_ind]) for x in all_data])))
    if n_bucket > len(lengths):
        n_bucket = len(lengths)

    splits = np.array(lengths)[
        (np.linspace(0, 1, 5, endpoint=False) * len(lengths)).astype(int)
    ].tolist()

    splits += [np.inf] 

    if debug:
        print(splits)

    ind_data = {}
    for x in all_data:
        l = len(x[bucket_ind])
        for ind, s in enumerate(splits[:-1]):
            if l >= s and l <= splits[ind + 1]:
                if ind not in ind_data:
                    ind_data[ind] = []
                ind_data[ind].append(x)
                break

    inds = sorted(list(ind_data.keys()))
    ind_p = [len(ind_data[x]) / len(all_data) for x in inds]
    if debug:
        print(np.sum(ind_p), ind_p)

    if isinstance(ws, (list, tuple)):
        assert len(ws) == len(data), 'The lengh of word sequence is not equal to the length of data while word sequence is list or tuple'

    if isinstance(add_end, bool):
        add_end = [add_end] * len(data)
    else:
        assert(isinstance(add_end, (list, tuple))), "add_end must be bool, list or tuple"
        assert len(add_end) == len(data), "The length of add_end should be equal as data when it is not bool"

    mul = 2
    if raw:
        mul = 3

    while True:
        choice_ind = np.random.choice(inds, p=ind_p)
        if debug:
            print('choice_ind', choice_ind)
        data_batch = random.sample(ind_data[choice_ind], batch_size)
        batches = [[] for i in range(len(data) * mul)]

        max_lens = []
        for j in range(len(data)):
            max_len = max([
                len(x[j]) if hasattr(x[j], '__len__') else 0
                for x in data_batch
            ]) + (1 if add_end[j] else 0)

            max_lens.append(max_len)

        for d in data_batch:
            for j in range(len(data)):
                if isinstance(ws, (list, tuple)):
                    w = ws[j]
                else:
                    w = ws

                line = d[j]
                if add_end[j] and isinstance(line, (tuple, list)):
                    line = list(line) + [WordSequence.END_TAG]

                if w is not None:
                    x, xl = transform_sentence(line, w, max_lens[j], add_end[j])
                    batches[j * mul].append(x)
                    batches[j * mul + 1].append(xl)
                else:
                    batches[j * mul].append(line)
                    batches[j * mul + 1].append(line)
                if raw:
                    batches[j * mul + 2].append(line)
        batches = [np.asarray(x) for x in batches]

        yield batches